# Introduction


Autophot is used to create AGN lightcurves from multiple telescopes to then be inter-calibrated using pyCALI. It is intended that the user have control with modular options and seperated functions.  

# Installations

I would not recommend using a windows system for some certain programs/packages that are required for the later part of the process. That said general photometry can easily be performed with any python 3 installation such as with conda. I would recommend using a linux distro especially for compatibility with pyCALI. If you do not have a linux distro installed a good alternative is a Virtual Machine (VM). VirtualBox is a great virtual machine program.

When setting up your VirtualBox VM it can be any linux distro, but it is best to simply use the latest stable ubuntu. Make sure to set your memory settings to an adequate level for the VM. 

## Anaconda
To start install the open source program Anaconda (https://www.anaconda.com/products/individual) onto your machine, if you do not have it, and create a **python 3** installation. 

### Environment

Here is a link to conda environment information that I am pulling from https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

Anaconda has a nice feature of allowing for seperate installations of packages with environments, this is handy, because with constant updates to packages certain dependencies may fail when updated then your whole code doesn't work. Environment keeps the packages together to work together.

You can create an environment with 

conda env create enviro_name

You can check if it was created

conda env list

---
## pyCALI

https://pycali.readthedocs.io/en/latest/

https://github.com/LiyrAstroph/PyCALI

Here is the github link to pyCALI, there are instructions and documentation included. I will go through the major steps and various installations needed to get pyCALI running later. 


---

To load in the pyautophot package

In [1]:
from pyautophot import * 

Hello World!


In [2]:
# this is to quiet certain astropy warnings that come out as the code runs (they are usually about deprecated functions)
import warnings

Before we get into any use of the actual programs there is a short laundry list that you should check in order for this to run smoothly 

# Checking Your Data

The package has been made with images from multiple telescopes in mind. A general method of organizing data is creating a directory that contains the images from each different telescope which may look like.. 
- Telescopes
  - LCO
    - g
    - r
    - i 
    - z
  - Zowada
    - u
    - g
    - r
    
This is assuming that all the images have been through some reduction pipeline (flat fielded, dark subtraction, etc.)

You will also need the following for each telescope, which are typically found in the FITS headers (and are static variables)
- Name (your chosen label)
- gain (in DN/e-)
- read noise (in e-)
- pixel scale (in arcsec/pix)

Additionally you will need a list of RA and DEC in degrees for the AGN of interest and RA and DEC of comparisons stars to correct for the differences in nights imaging. Down below you'll find descriptions and examples of how to use the functions called. 

# Functions\Callables 




- **phot**(*directory=None,outdir=None,coords=None,img_filter=None, target_name=None, snr_cutoff=0, telname=None, rdnoise=0., pixsc=1., gain=1., rad=[5.,15.,20.,12.], sky_method='DAOphot', sky_hist=False, img_hist=False, renew=True, naver_on=False, centroid_show=False, c_func='com'*)

  Performs photometry on a directory of FITS files given a set of comparison stars and AGN. Calls on astropy's photutils package primarily. 
  - **directory**: *str* instance\
    The file directory where the images of a single filter are found
  - **output_directory**: *str* instance\
    Where all files produces will be stored.
  - **coordinates**: *array_like* instance\
    The AGN, and comparison star RA,DEC given as a list/array
  - **img_filter**: *str* instance\
    The image filter of said image directory
  - **target_name**: *str* instance\
    The AGN's name
  - **SN_cutoff**: *float* instance\
    A signal-to-noise ratio cutoff for photometry, default value is set to 0 which will keep all non-negative photometric measurements.
  - **telname**: *str* instance\
    The telescopes label that you choose, for LCO telescopes use lco as the label as lco always has specific headers that can be used.
  - **rdnoise**: *float* instance\ 
    Readnoise of the telescope given in e- units, generally it can be found in the header, but not always. If you are using the lco label you do not need to input this.
  - **pixsc**: *float* instance\
    Pixel scale of the telescope in units of arcsec/pix, similarly to read noise it may be found in the header. If you are using the lco label you do not need to input this.
  - **gain**: *float* instance\
    Gain of the telescope in units of DN/e-. If you are using the lco label you do not need to input this.
  - **rad**: *array_like* instance\
    An array of arcsecond values to be used for the aperture, inner and outer annulus, and sky box in the photometry. The sky box is the size of box that will be used to find the centroid of an object.
  - **sky_method**: *{'DAOphot','default'}, optional*\ 
    There are currently two methods for sky background estimation, one is an annulus median, the second is similar to the DAOphot sky mmm.pro method.
      - 'DAOphot': 3*median - 2*mean if mean > median
      - 'clipped media': 3-sigma clipped median
  - **sky_hist**: *Boolean* instance, optional\
    A simple diagnostic plot that will show a histogram of the sky annulus ring's pixel values.
  - **img_hist**: *Boolean* instance, optional\
    Like sky_hist, but for the entire image, shown in log scale.
  - **renew**: *Boolean* instance, optional\
    When set to False it will check if there exists an output file for the image it is currently working on and will skip it. This is useful when you are downloading images on a weekly/repeated basis and you need to run the photometry on only new images.
  - **naver_on**: *Boolean* instance, optional\
    This is specific for images that have been averaged and need to have their uncertainty adjusted for said co-addition.
  - **centroid_show**: *Boolean* instance, optional\
    Simple plot that will show the current image with centroids and intial RA DEC values inputted to ensure the centering is working.
  - **MPC**: *dict* instance\
    Dictionary containg the Minor Planet Center (MPC) codes of telescopes being used (this dictionary may also be self-defined if certain observatories don't have codes). Ex. {'Hubble':'250','Faulkes North 2m':'F65'}, see https://minorplanetcenter.net/iau/lists/ObsCodesF.html for codes
  - **c_func** *string* instance\
    Centroid function to be used for centering default is center of mass, uses Astropy centroiding library.
      - 'com': center of mass
      - '2dg': 2-dimensional gaussian
  
- **phot_to_one**(outdir,img_filter,coords)
 
  Will go through the photometry csvs and create telescope and object specific files of the photometry. 
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **img_filter**: *str* instance\
    Image filter.
  - **coords**: *array_like* instance\
    The AGN, and comparison star RA,DEC given as a list/array.
 
- **scale_AGN**(outdir,img_filter,target_name,who=[],clean=False)

  Takes the nightly scaling factors that are calculated through the fit() function and applies it to the AGN lightcurve and saves it.
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **img_filter**: *str* instance\
    Image filter.
  - **coords**: *array_like* instance\
    The AGN, and comparison star RA,DEC given as a list/array.
  - **target_name** *string* instance\
    Name of AGN to be used for the filename.
  - **who** *array_like* instance, optional\
    An MPC label of telescope(s) you want to apply the scaling too, if not specified scaling will be applied to all available telescopes.
  - **clean** *Boolean* instance\
    Mister clean mister clean

- **plot_scaled**(outdir,img_filter,target_name,cdict={},who=[],filename=False)

- **plot_aligned**(outdir,img_filter,target_name,reference=None,cdict={},all_black=False)

- **clean_lc**(outdir,img_filter)

- **condense**(outdir,img_filter,who=[],delta=0.25)

- **plot_condense**(outdir,img_filter,target_name,cdict={},mpc={},who={},multi=True,single=False,counts=False)

- **excess_var**(outdir,img_filter,starlist,who=[])

- **cali_format**(outdir,flt,target_name,reference=None,condense=True,pycali_datadir=None,raw=False,mjd=False)

- **fit**(outdir,coords,img_filter,who=[])

  A chi squared minimization routine to find the optimal scale factors for star normalization using the set of comparison stars. Scale is telescope specific and will be saved in a csv. 
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **coords**: The AGN, and comparison star RA,DEC given as a list/array
  - **img_filter**: *str* instance\
    Image filter to perform the fit on.
  - **who**: *array_like* instance, optional\
    For use if you want to only perform the fitting on specific telescopes using the user defined label names.
  
- **star_norm**(outdir,coords,img_filter,who=[],clean=False)
  
  Will apply the scale to normalize the comparison stars and save them for late use in flux conversion. 
  - **outdir**: *str* instance\
     Where all files produces will be stored.
  - **coords**: The AGN, and comparison star RA,DEC given as a list/array
  - **img_filter**: *str* instance\
    Image filter where the stars will be normalized.
  - **who**: *array_like* instance, optional\
    For use if only speicific telescopes are to have their stars normalized (to avoid repetition).
  - **cutoff**: *float* instance\
    A standard deviation cutoff for the **normalized** stars, the default is set to 3$\sigma$.
  - **cut**: *Boolean* instance\
    Whether to perform the sigma cutoff on the normalized stars.
  
- **plot_normed**(outdir,img_filter,coords,cdict={},mpc={},who=[])

  A comparison plot of the raw comparison stars and normalized comparison stars. 
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **img_filter**: *str* instance\
    Image filter to plot.
  - **coords**: *array_like* instance\
    The AGN, and comparison star RA,DEC given as a list/array
  - **cdict**: *dictionary* instance\
    A dictionary for the telescopes and the colors you would like them to correspond to.
  - **mpc**: *dictionary* instance\
    Minor planet center dictionary.
  - **who**: *array_like* instance\
    For use if only specific telescopes should be plotted. 
  

  
- **true_mag_retrieve**(outdir,tar=None,mags=None,who=None, mags_tags=None,feedback=False,f_to=[])
  
  Performs the flux conversion from counts to cgs units by using the normalized comparison stars. Using the comparison stars true magnitudes, the function finds the average scale factor for the flux conversion with a sigma clipped values. 
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **who**: *array_like* instance, optional\
  
  - **mags_tags**: *array_like* instance\
  
  - **feedback**: *Boolean* instance, optional\
  
  - **f_to**: *array_like* instance, optional\
  
- **plot_cgs**(cdict={},whoo=None,outdir=None,mpc={},who=[],f_to=[],multi=True,single=False)

  Plots the flux converted AGN light curves in a multi plot. 
  - **cdict**: *dictionary* instance\
  
  - **target_name**: *str* instance\
  
  - **outdir**: *str* instance\
    Where all files produces will be stored.
  - **mpc**: *dictionary* instance\
  
  - **who**: *array_like* instance, optional\
  
  - **f_to**: *array_like* instance, optional\
  
  - **multi**: *Boolean* instance, optional\
  
  - **single**: *Boolean* instance, optional\
  
    
- **error_ratio**(outdir,coords,img_filter)

- **plot_multicali**(tn,cdict,avg=False)

- **plot_cali**(tn,cdict,calidir,filters=[])

- **pycali_run**(filt='g', run=1, outdir='')

- **produce_clean**()
  

# Example: Mrk 817

First you'll want to start with a directory to pull from and out directory to output and other things we can outright define (**supply zip file**)

In [ ]:
# Input the directory where your images are stored
directory = '/home/korbinite5646/AGN_home/FITS/'
# Here you should input a directory where the photometry will be stored along with the other data products
# I would suggest making a seperate directory
outdirectory = '/home/korbinite5646/AGN_home/MRK817/notebook/'
AGN_name = 'Mrk817'
img_filter = 'g'

You'll also need to go through the images and get comparison stars. You may find the AGN's RA, DEC through things like SIMBAD. Generally you will want > 5 comparison stars for good statistics. Note that the first set of RA DEC you give is the AGN!

In [ ]:
coords = [ [219.09200836,58.7942765], [219.1357,58.83312] , [219.17719,58.84183] , [219.150785,58.766423] ,[218.6666,58.63422]  , [219.009178,58.875326], [219.077094,58.677452], [219.05994,58.666196]]

Another thing to keep create are the MPC code dictionaries and your given plot color dictionary. You'll only have to define this once and you can freely add telescopes to it as your project progresses.

In [ ]:
cdict = { 'zowada': 'blue', 'F65': 'saddlebrown', 'V39': 'darkorange', 'V37': 'darkgreen', 'W85': 'orchid', 'W85': 'indigo', 'W87': 'indianred', 'K93': 'olive', 'K91': 'darkorange', 'K92': 'y', 'Q63': 'sienna', 'Q64': 'chocolate','WISE': 'lightblue', 'liverpool': 'red'}
MPC ={'ogg-clma-2m0a': 'F65', 'ogg-clma-0m4b': 'T04', 'ogg-clma-0m4c': 'T03', 'elp-doma-1m0a': 'V37', 'elp-domb-1m0a': 'V39', 'elp-aqwa-0m4a': 'V38', 'lsc-doma-1m0a': 'W85', 'lsc-domb-1m0a': 'W86', 'lsc-domc-1m0a': 'W87', 'lsc-aqwa-0m4a': 'W89', 'lsc-aqwb-0m4a': 'W79', 'cpt-doma-1m0a': 'K91', 'cpt-domb-1m0a': 'K92', 'cpt-domc-1m0a': 'K93', 'cpt-aqwa-0m4a': 'L09', 'coj-clma-0m4a': 'Q58', 'coj-clma-0m4b': 'Q59', 'coj-doma-1m0a': 'Q63', 'coj-domb-1m0a': 'Q64', 'coj-clma-2m0a': 'E10','zowada':'zowada'}

We have everything to run the first step of photometry code! Note that this is from Las Cumbres Observatory, which is a network of telescopes, but their headers are uniform. So, calling telname 'lco' it will search the header for the relevant parameters, hence not giving rdnoise, pixsc or gain.

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
phot(directory=directory,
    outdir=outdirectory,
    coords=coords,
    img_filter=img_filter,
    target_name=AGN_name,
    telname='lco'
    )

Now we have photometry done on one telescope and you should be able to see csv files of it in your directory! Well what if we want to do a bunch of filters, since this is made for multi-wavelenght campaigns, and a bunch of different telescopes. We can work this out with just some bit of code.

In [ ]:
# The LCO images that we have come in ugriz and BV. Lets start with a list of the filters we'll use.
img_filters = ['B','g','r']
# The directory where the images will be found
directory = ''

# with that we can loop through the filters to do all that photometry, depending on how many images you're working with this may take a few minutes. 
for img_f in img_filters:
    phot(directory=directory+'/'+img_f,
        outdir=outdirectory,
        coords=coords,
        img_filter=img_f,
        telname='lco'
        )

Go ahead and take a look at your output directory it should look something like this now...

*image here*

And below is what the tables for the photometry output looks like

In [ ]:
# open csv file with pandas
df = pd.read_csv()
print(df)

Let's repeat the above step, but now for a different telescope. The snippet of code is the same, except for the parameters.

In [ ]:
zowada_filters = ['g','r']
directory = ''
for z in zowada_filters:
    phot(directory=directory+'/'+z,
        outdir=outdirectory,
        coords=coords,
        img_filter=z,
        telname='zowada',
        rdnoise=8.0,
        pixsc=0.882,
        gain=0.476)

At this point we have enough to start creating light curves. First we'll have to take all the csv's that have been created and format them. This will take all the telescopes that you have run phot() on and put them into one.

In [ ]:
# I'll be calling it on just the g band
phot_to_one(outdir=outdirectory,
           img_filter='g',
           coords=coords,
           mpc=MPC);

A chunkier step is the chi square minimization of the comparison stars. With a set of comparison stars we'll use 

$$\chi^2 = \Sigma_n\Sigma_i \frac{(f(n)F(n)-\bar{F(i)})^2}{f(n)\epsilon^2}$$

Where n is the number of images/nights and i is the number of comparison stars. Minimizing this function with free parameter f(n) will give us scale factors for each night to correct for the varying conditions. This in turn will be applied to the AGN's light curve.

In [ ]:
fit(outdir=outdirectory,
   coords=coords,
   img_filter='g'
   );

Next we apply the scale factor to the stars, because it will be used later in the flux conversion.

In [ ]:
star_norm(outdir=outdirectory,
         coords=coords,
         img_filter='g'
         );

Let's visually check how the fitting worked out, what you are looking for are relatively flat plots with scatter. If you notice anything anomalous then it would be a good idea to look at the images. In this case take a look at the csv files as they have attached file names to the HJD dates.

In [ ]:
normed_plots(outdir=outdirectory,
            img_filter='g',
            coords=coords,
            cdict=cdict,
            mpc=MPC,
            )

Nothing seems unusual in the comparison star light curves, and we see some expected scatter as we aren't going for a perfect fit. 

We can work on the AGN now itself. First we'll apply the scale and save that data, and afterwards plot the lightcurves per telescope. The scale_plot() function will plot the individual telescopes. In the process of making the lightcuve tables there are some rows we can clean up, calling clean_lc() will get rid of any bad values (NaN, inf, etc.) and sort it with HJD descending.

In [ ]:
scale_plot(outdir=outdirectory,
          coords=coords,
          img_filter='g',
          target_name=AGN_name,
          cdict=cdict,
          mpc=MPC)
clean_lc(outdir=outdirectory,
        img_filter='g')

While we have them individually you also have the option to over lay them over each other. They aren't on the same flux scales, so they are brought close by some factor of the flux means according to a reference telescopes.

In [ ]:
dumb_align(outdir=outdirectory,
           img_filter='g',
           coords=coords,
           target_name=AGN_name,
           reference='V37',
           cdict=cdict)

It looks like the photometry is doing well and the general shapes of the light curves are agreeing.

The flux conversion step takes the normalized comparison stars and their true magnitudes in various filters to find an average scale factor that will appropiately give us flux wavelength units. After that we'll plot the physical units lightcurves, the light curves should be fairly near each other. If you find that one of the telescopes is not close then double check the magnitudes you put in for your comparison stars.

In [ ]:
true_mag_retrieve(outdir=outdirectory,who='Mrk817',f_to=['g'])
# no print statements or plots are produced here - should add in print statements to let user know what has been done 

In [ ]:
cgs_plot(cdict=cdict,outdir=outdirectory,mpc=MPC,f_to=['g'],whoo=AGN_name,single=True)

Generally telescopes take multiple images per night, so we have the option to create a weighted average light curve within some $\Delta t$. 

In [ ]:
condense(outdirectory,'g')

In [ ]:
# Need to allow for single plots again
plot_condense(outdirectory,'g',AGN_name,cdict,MPC)

Close to our last steps now - all that is left to do is feed the light curves to Pycali, which requires a specific format. We do that by using cali_format(), you have a choice to pick a reference telescope for a filter, generally you want to pick the light curve that spans date range well and is fairly good quality. Within the example we're doing the LCO telescopes V37 spans the largest range and has low uncertainties. 

In [ ]:
# Need to fix it to allow for non averaged
cali_format(outdirectory,'g',AGN_name)

# Template Code

In [ ]:
coords = 
target_name = 
outdir = 

cdict = {}
MPC = {}

In [ ]:
a_filters = []
a_direc = ''
for a_f in a_filters:
    phot(a_direc+a_f,
         outdir,
         coords=coords,
         img_filter=a_f,
         target_name=target_name,
         telname='a_tel',
         cutoff=10.)

In [ ]:
b_filters = []
b_direc = ''
for b_f in b_filters:
    phot(b_direc+b_f,
         outdir,
         coords=coords,
         img_filter=b_f,
         target_name=target_name,
         telname='b_tel',
         cutoff=10.)

In [ ]:
c_filters = []
c_direc = ''
for c_f in c_filters:
    phot(c_direc+c_f,
         outdir,
         coords=coords,
         img_filter=c_f,
         target_name=target_name,
         telname='c_tel',
         cutoff=10.)

In [ ]:
img_filters = []
for img_filter in img_filters:
    phot_to_one(outdir, img_filter=img_filter, coords=coords, mpc=MPC)
    fit(outdir,coords,img_filter)
    star_norm()
    normed_plots()
    scale_plot()
    clean_lc()
    dumb_align()

In [ ]:
true_mag_retrieve()

In [ ]:
cgs_plot()

In [ ]:
for img_filter in img_filters:
    condense()
    plot_condense()

In [ ]:
cali_format()

# Diagnostics

What to do when the code isn't working or something does not look right. There are multiple optional parameters in the functions called made for this reason we'll explore each one to see what they tell us.

## Photometry - Image and Annulus Histograms

The phot() function has two optional diagnostics, the first being an image histogram, and then the annulus' histogram. To enable either of them your call should look like this

``` Python
phot(img_hist=True,sky_hist=True)
```

Down below we'll run it on some different telescopes.

In [ ]:
phot('E:/AGN_central/mrk817/notebook',outdir=outdirectory,img_filter='g',img_hist=True,coords=coords,telname='lco')

# add image title!

When sky_hist is run it will create a sky annulus histogram for each coordinate pair given.

In [ ]:
# add coords and image title!
phot('E:/AGN_central/mrk817/notebook',outdir=outdirectory,img_filter='g',sky_hist=True,coords=coords,telname='lco')

In [ ]:
# excess variance?

In [ ]:
# true mag thing